In [ ]:
# This Python 3 environment comes with many helpful analytics libraries installed
# It is defined by the kaggle/python Docker image: https://github.com/kaggle/docker-python
# For example, here's several helpful packages to load

import pandas as pd # data processing, CSV file I/O (e.g. pd.read_csv)

# Input data files are available in the read-only "../input/" directory
# For example, running this (by clicking run or pressing Shift+Enter) will list all files under the input directory

import os
for dirname, _, filenames in os.walk('/kaggle/input'):
    for filename in filenames:
        print(os.path.join(dirname, filename))

# You can write up to 20GB to the current directory (/kaggle/working/) that gets preserved as output when you create a version using "Save & Run All" 
# You can also write temporary files to /kaggle/temp/, but they won't be saved outside of the current session

In [ ]:
train = pd.read_csv('/kaggle/input/store-sales-time-series-forecasting/train.csv')
print(train.head())
print(train.info())
print(train.describe())

In [ ]:
holidays_events = pd.read_csv('/kaggle/input/store-sales-time-series-forecasting/holidays_events.csv')
print(holidays_events.head())
print(holidays_events.info())
print(holidays_events.describe())

In [ ]:
holidays_events['type'].unique()
#holidays_events['locale_name'].unique()

In [ ]:
he_neve = holidays_events[holidays_events['type']!='Event']

In [ ]:
import matplotlib.pyplot as plt

plt.figure(figsize=(10,10))
plt.hist(holidays_events['type'])
plt.show()

In [ ]:
oil = pd.read_csv('/kaggle/input/store-sales-time-series-forecasting/oil.csv')
print(oil.head())
print(oil.info())
print(oil.describe())
print(oil.isna().sum())

In [ ]:
from sklearn.impute import KNNImputer
knn_imputer = KNNImputer()
oil_out = oil.copy()
oil_out['dcoilwtico']=knn_imputer.fit_transform(oil[['dcoilwtico']])

print(oil.info())
print(oil_out.info())

In [ ]:
sample_sub = pd.read_csv('/kaggle/input/store-sales-time-series-forecasting/sample_submission.csv')
print(sample_sub.head())
print(sample_sub.info())

In [ ]:
stores = pd.read_csv('/kaggle/input/store-sales-time-series-forecasting/stores.csv')
print(stores.head())
print(stores.info())

In [ ]:
transaction = pd.read_csv('/kaggle/input/store-sales-time-series-forecasting/transactions.csv')
print(transaction.head())
print(transaction.info())

In [ ]:
train2 = pd.merge(train, oil_out, on='date', how='left')
print(train2.info())

In [ ]:
he_neve['type'].unique()

holiday_types = ['Holiday', 'Transfer', 'Additional', 'Bridge']

is_holiday_cond = (he_neve['type'].isin(holiday_types)) & (~he_neve['transferred'])

he_neve['is_holiday'] = is_holiday_cond.astype(int)

#he_neve.groupby('date').sum()

In [ ]:
he_unique = he_neve.groupby('date')['is_holiday'].max().reset_index()

print(he_unique)

In [ ]:
train3 = pd.merge(train2, he_unique, on='date', how='left' )
print(train3.info())

In [ ]:
he_neve

In [ ]:
he_neve[(he_neve['type'] == 'Transfer') & (he_neve['transferred'])]

In [ ]:
old_dates = set(train2['date'])
new_dates = set(train3['date'])

In [ ]:
print(len(old_dates))
print(len(new_dates))

In [ ]:
train3.isna().sum()

In [ ]:
train3['dcoilwtico']

In [ ]:
test = pd.read_csv('/kaggle/input/store-sales-time-series-forecasting/test.csv')
print(test.head())
print(test.info())

In [ ]:
train3

In [ ]:
test

In [ ]:
from sklearn.impute import KNNImputer
from functools import reduce
# Data integrity
# load data
data_dict = {}
for dirname, _, filenames in os.walk('/kaggle/input'):
    for filename in filenames:
        if filename.endswith('.csv'):
            data_dict[filename.split('.')[0]] = pd.read_csv(os.path.join(dirname,filename))
        else:
            pass

# preprocessing holidays
he = data_dict['holidays_events'][data_dict['holidays_events']['type'] != 'Event']
holiday_types = ['Holiday', 'Transfer', 'Additional', 'Bridge']
is_holiday_cond = (he['type'].isin(holiday_types)) & (~he['transferred'])
he['is_holiday'] = is_holiday_cond.astype(int)
he = he.groupby('date')['is_holiday'].max().reset_index()

data_dict['holidays_events'] = he

del he

# imputing oil NaN
knn_imputer = KNNImputer()
oil = data_dict['oil']
oil['dcoilwtico'] = knn_imputer.fit_transform(oil[['dcoilwtico']])

data_dict['oil'] = oil

del oil

# merging store location information & store statistic information
transactions = data_dict['transactions']
stores = data_dict['stores']

store_stats = transactions.groupby('store_nbr')['transactions'].agg(['mean', 'std']).reset_index()
store_stats.columns = ['store_nbr', 'avg_transactions', 'std_transactions']

store_profile = pd.merge(stores, store_stats, on='store_nbr', how='left')

# make final train data

train1 = reduce(lambda x, y: pd.merge(x, y, on='date', how='left'), [train, data_dict['oil'], data_dict['holidays_events']])

final_train = pd.merge(train1, store_profile, on='store_nbr', how='left')

del train1

In [ ]:
final_train.info()

In [ ]:
from sklearn.preprocessing import LabelEncoder
# value Encoding
label_encoder = LabelEncoder()
final_test = final_train.copy()
cat_columns = final_train.select_dtypes('object').columns.tolist()
del cat_columns[0]
for column in cat_columns:
    final_test[column] = label_encoder.fit_transform(final_test[[column]])


final_test

In [ ]:
final_train.select_dtypes('object').columns

In [ ]:
store_profile

In [ ]:
data_dict['oil']

In [ ]:
data_dict['holidays_events']

In [ ]:
pd.merge(test, data_dict['oil'], on='date', how='left').isna().sum()

In [ ]:
final_train.isna().sum()

In [ ]:
temp = final_train[['date', 'dcoilwtico']]
temp[temp.isna().any(axis=1)]

In [ ]:
from sklearn.preprocessing import MinMaxScaler
knn_imputer = KNNImputer()
temp = final_train[['date', 'dcoilwtico']]
temp['date_numeric'] = pd.to_datetime(temp['date']).map(pd.Timestamp.to_julian_date)
temp2 = temp[['date_numeric', 'dcoilwtico']]

scaler = MinMaxScaler()
temp_scaled = scaler.fit_transform(temp2)

temp_imputed = knn_imputer.fit_transform(temp_scaled)

temp2['dcoilwtico'] = scaler.inverse_transform(temp_imputed)[:, 1]
temp2